faiss事前準備:

miniconda_python(3.10版本):

https://docs.conda.io/projects/miniconda/en/latest/miniconda-other-installer-links.html

裝完打開anaconda prompt:

做一個環境```conda create -n VScode-env-3.10.13 python=3.10.13```

做完啟動```conda activate VScode-env-3.10.13```

安裝faiss插件

```conda install -c pytorch faiss-cpu=1.7.4 mkl=2021 blas=1.0=mkl```

```conda install -c pytorch/label/nightly faiss-cpu```

關掉prompt 重新開啟VScode 右上角kernel 選擇another kernel->python env->VScode-env-3.10.13

VScode商城:

python

jupyter

import和拿key

沒裝openai_SDK使用:```pip install openai```

沒裝tabulate使用:```pip install tabulate```

沒裝pandas:```pip install pandas```

沒設定key使用:```setx OPENAI_API_KEY “<yourkey>”```

```echo %OPENAI_API_KEY%```可以看你的KEY

In [65]:
import os
import time
from openai import OpenAI
client = OpenAI()
OpenAI.api_key = os.getenv('OPENAI_API_KEY')

# input在此輸入

使用同一個Assistant和Thread從這邊開始run (記得把最後刪除Assitant的code註解掉)

In [66]:
input = "Can I get deleted account back? How to adjust the break time after planting a tree?"

前處理 問題建議(要回答)還是抱怨#或誇獎@(不回答)

In [67]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": "you are classify bot. you are prohibited answering question.here is three rule:1.if sentence is a compliment, you must only answer with '@'2.if sentence is a complain, you must only answer with '#'3.if sentence is not compliment nor complain, you must summarise and grammerly complete user's sentence then output"},
    {"role": "user", "content": input}
  ]
)
time.sleep(5)
prequiz = completion.choices[0].message.content
if prequiz =="#":
    prequiz = "complain"
    print(prequiz)
elif prequiz =="@":
    prequiz = "compliment"
    print(prequiz)
else:
    print("\""+prequiz+"\" is this your question?")

"Can I get my deleted account back? How can I adjust the break time after planting a tree?" is this your question?


In [68]:
print(completion.choices[0].message.content)

Can I get my deleted account back? How can I adjust the break time after planting a tree?


# channel在此輸入(1.App 2.Email 3.comment)

In [69]:
channel = 1

# instruction在此輸入

In [70]:
instruction_App = "1.You are InApp chatting bot from Flora customer service. 2.When user ask a question, You must reply based on 3 history questions and answers. 3.If 3 history questions and answers not related to my question, you must reply only a special mark:'*'."
instruction_App_check = "1.You are InApp chatting bot from Flora customer service. 2.When user ask a question, You must reply based on 3 history questions and answers. 3.If 3 history questions and answers not related to my question, you must reply only a special mark:'*'. 4.when you encounter question concerning email, you must check email fitting RFC5322 standard or not. if not, ask again"
instruction_Email = "You are email reply bot from Flora customer service. You must answer based on the history questions and answers from Flora's database. If there is 0 history question and answer, you must reply only a special mark:'*'"
instruction_Comment = "You are Apple Appstore comment reply bot from Flora customer service. You must answer based on the history questions and answers from Flora's database. If there is 0 history question and answer, you must reply only a special mark:'*'"

assistant1負責掌握多少問題和進度

In [71]:
still = 1
def get_still():
    return still
def set_still_to_zero():
    global still
    still =0
    return
init = 0
def get_init():
    return init
def set_init_to_one():
    global init
    init = 1
    return

In [72]:
assistant1 = client.beta.assistants.create(
    name="sentence queue",
    instructions="you are a sentence storing queue. you must not modify any sentence and you can perform some action listed below: 1.'enqueue':You must add sentence to queue.When encounter multiple question,split into several questions then 'enqueue' ono by one. 2.'dequeue': output one sentence then remove one sentence from queue. when there is no sentence in queue,you must output only'$'.",
    model="gpt-3.5-turbo-1106"
)
thread1 = client.beta.threads.create()

In [92]:
if get_init() == 0:
  message1 = client.beta.threads.messages.create(
    thread_id=thread1.id,
    role="user",
    content= "enqueue \""+prequiz+"\"\ndequeue"
  )
  run1 = client.beta.threads.runs.create(
    thread_id=thread1.id,
    assistant_id=assistant1.id
  )
  time.sleep(5)
  input = client.beta.threads.messages.list(thread_id=thread1.id,order='desc').data[0].content[0].text.value
  print(input)
  set_init_to_one()
elif get_init() == 1:
  if get_still() ==1:
    message1 = client.beta.threads.messages.create(
      thread_id=thread1.id,
      role="user",
      content= "dequeue"
    )
    run1 = client.beta.threads.runs.create(
      thread_id=thread1.id,
      assistant_id=assistant1.id
    )
    time.sleep(5)
    input = client.beta.threads.messages.list(thread_id=thread1.id,order='desc').data[0].content[0].text.value
    if input =="$":
      set_still_to_zero()
      print("nothing in queue1")
    else:
      print(input)
  elif get_still() == 0:
    print("nothing in queue2")

How can I adjust the break time after planting a tree?


In [93]:
m=client.beta.threads.messages.list(thread_id=thread1.id,order='asc')
for i in m.data:
  print(i.content[0].text.value)
  print("\n")

How can I adjust the break time after planting a tree?


dequeue


"Can I get my deleted account back?"



enqueue "Can I get my deleted account back? How can I adjust the break time after planting a tree?"
dequeue




In [94]:
instruction = ""
if channel == 1 :
    instruction = instruction_App
elif channel == 2 :
    instruction = instruction_Email
elif channel == 3 :
    instruction = instruction_Comment

instruction += "\nYou must follow the SOP: \n"

# The rule that Assistant should follow
SOP = [
    "1. Based on history Q&A to answer.",
    "2. If the customer's problem still can not be solved after you replied, reply only a special mark:'*'.",
]

for i in SOP:
    instruction = instruction + i + "\n"

print(instruction)

先準備faiss

讀embeddings資料

並把dataframe轉為numpy

In [95]:
import pandas as pd
import numpy as np

csv_file = 'data\embeddingsV3_3072.csv'
df = pd.read_csv(csv_file)
data = df.to_numpy()
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,-0.030886,-0.020970,0.002210,-0.032358,-0.014908,-0.007581,0.005008,0.012017,-0.034472,0.018240,...,0.001131,0.019792,-0.011917,0.007220,-0.018026,0.003841,0.001450,0.020649,-0.003747,0.021291
1,0.013026,0.013851,-0.009579,-0.059783,0.008046,-0.015229,0.002933,0.004625,-0.002641,0.046384,...,0.017729,0.013825,0.001612,-0.019378,0.011802,-0.009483,-0.006945,0.005122,0.004068,0.009393
2,-0.005118,-0.024766,-0.014762,-0.029704,-0.016605,-0.011532,-0.001856,0.039089,0.005199,0.034809,...,0.013769,0.028311,0.001599,-0.015600,0.024998,-0.024624,-0.000467,-0.005411,0.007058,0.008451
3,-0.019629,0.002125,-0.002155,-0.013240,0.015541,-0.034832,-0.044084,0.013315,0.035132,0.020191,...,0.003723,0.032956,-0.020729,-0.005295,0.016266,0.004657,-0.020029,0.008870,0.011921,0.033432
4,0.005265,0.015462,0.002796,-0.057215,0.013133,-0.032900,-0.001634,0.008408,-0.005457,0.024303,...,0.020498,0.016450,0.004387,-0.020389,-0.004006,0.007847,0.003600,0.005015,0.010267,0.010481


確認內容

In [96]:
print(data.shape[0])
print(type(data))
print(data.dtype)
print(data[0][0])

669
<class 'numpy.ndarray'>
float64
-0.0308859776705503


寫入維度 並把data加入faiss的index

In [97]:
import faiss

index = faiss.IndexFlatL2(data.shape[1])
index.add(data)

創assistant,thread!

In [98]:
from tabulate import tabulate

assistant2 = client.beta.assistants.create(
    name="Flora's Customer Service",
    instructions=instruction,
    model="gpt-3.5-turbo-1106"
)

thread2 = client.beta.threads.create()

做問題的embedding

TOP-k的設定並查找問題

拿答案

In [99]:
def get_similar(input):
    response = client.embeddings.create(input=input, model="text-embedding-3-large", dimensions=3072)
    question = np.array(response.data[0].embedding, ndmin=2)
    k = 3
    distances, indices = index.search(question, k)
    csv_question = 'data\manual.csv'
    qdf = pd.read_csv(csv_question)
    qarray = qdf.to_numpy()
    answer = [(qarray[i], float(dist)) for dist, i in zip(distances[0], indices[0])]
    history = ""
    Qcount = 0
    for i in range(0,k):
        Qcount += 1
        history = history +str(i+1)+". "+ answer[i][0][0]+"\n"+answer[i][0][1]+"\n"
    return Qcount, history

距離設定dist內才顯示

# Prompt內容在此

In [100]:
similar = get_similar(input)
prompt = "Here is "+str(similar[0])+" history questions and answers similar to this question according to Flora's database\n"+similar[1]+"Here is my question:\n"+input
print(prompt)

Here is 3 history questions and answers similar to this question according to Flora's database
1. Can I adjust the break timer after growing a tree?
You may adjust your break time after successfully growing a tree. Press the 'Timer' icon at the left of the 'See Stories' button on the green focus screen, then you will be able to adjust the break timer. When the timer is up, a 'Back to Work' notification will be sent to remind you to go back to work.
2. I want to change the resting time after I have planted a tree? 
You may adjust your break time after successfully growing a tree. Press the 'Timer' icon at the left of the 'See Stories' button on the green focus screen, then you will be able to adjust the break timer. When the timer is up, a 'Back to Work' notification will be sent to remind you to go back to work.
3. What if I need more time to finish my task after a tree grows up?
A focus session does not end when the tree grows. So, keep working! During this phase, the focus duration w

In [101]:
message2 = client.beta.threads.messages.create(
    thread_id=thread2.id,
    role="user",
    content= prompt
)

沒裝tabulate很醜

tabulate文件:https://pypi.org/project/tabulate/

表示assistant,thread,message內容

In [102]:
print(tabulate(assistant2,headers=['assistant','content'],tablefmt="simple_outline"))

┌─────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ assistant       │ content                                                                                                                                                                                                                                                 │
├─────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id              │ asst_pdVd0i9kFnLljlsxW5U8gwu9                                                                                                                                             

In [103]:
print(tabulate(thread2,headers=['thread','content'],tablefmt="simple_outline"))

┌────────────┬─────────────────────────────────┐
│ thread     │ content                         │
├────────────┼─────────────────────────────────┤
│ id         │ thread_Y6U2SrjvYCtQBkT0pyigaaqz │
│ created_at │ 1713890881                      │
│ metadata   │ {}                              │
│ object     │ thread                          │
└────────────┴─────────────────────────────────┘


In [104]:
print(tabulate(message2,headers=['message','content'],tablefmt="simple_outline"))

┌────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

這次的run

可以使用新instruction複寫assisstant裡的instruction!

like:

```
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)
```


In [105]:
run2 = client.beta.threads.runs.create(
  thread_id=thread2.id,
  assistant_id=assistant2.id
)

查看run的狀態!

以上如果還沒跑完會顯示queued!

以上如果跑完會顯示completed!

In [106]:
# run = client.beta.threads.runs.retrieve(
#   thread_id=thread.id,
#   run_id=run.id
# )
# print(tabulate(run,headers=['run','content'],tablefmt='simple_outline'))

while run2.status == "queued" or run2.status == "in_progress":
    run2 = client.beta.threads.runs.retrieve(
        thread_id=thread2.id,
        run_id=run2.id
    )
print(run2.status)
print(tabulate(run2,headers=['run','content'],tablefmt='simple_outline'))

completed
┌───────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ run                   │ content                                                                                                                                                                                                                                                 │
├───────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ id                    │ run_6rj2FuT18DFuSx7g1RYMzNzs                                                                                                            

In [107]:
# if function calling is required, print user input
try:
    print(run2.required_action.submit_tool_outputs.tool_calls[0].function.arguments[21:-2])

except AttributeError:
    print("No function calling")


No function calling


In [108]:
if run2.status == "requires_action":
    if run2.required_action.submit_tool_outputs.tool_calls[0].function.name == "get_history_question_and_answer":
        print("function calling: get_history_question_and_answer")
        similar = get_similar(run2.required_action.submit_tool_outputs.tool_calls[0].function.arguments[21:-2])
        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread2.id,
            run_id=run2.id,
            tool_outputs=[
                {
                    "tool_call_id": run2.required_action.submit_tool_outputs.tool_calls[0].id,
                    "output": similar[1]
                }
            ]
        )
else:
    print("No function calling")

No function calling


跑完後打開查看history messages!

In [109]:
time.sleep(5)
messages = client.beta.threads.messages.list(
  thread_id=thread2.id,
  order='asc'
)
for i in messages.data:
    print(tabulate(i,tablefmt='simple_outline'))

┌────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

訊息顯示

In [110]:
for i in messages.data:
  print(i.content[0].text.value)
  print("\n")

Here is 3 history questions and answers similar to this question according to Flora's database
1. Can I adjust the break timer after growing a tree?
You may adjust your break time after successfully growing a tree. Press the 'Timer' icon at the left of the 'See Stories' button on the green focus screen, then you will be able to adjust the break timer. When the timer is up, a 'Back to Work' notification will be sent to remind you to go back to work.
2. I want to change the resting time after I have planted a tree? 
You may adjust your break time after successfully growing a tree. Press the 'Timer' icon at the left of the 'See Stories' button on the green focus screen, then you will be able to adjust the break timer. When the timer is up, a 'Back to Work' notification will be sent to remind you to go back to work.
3. What if I need more time to finish my task after a tree grows up?
A focus session does not end when the tree grows. So, keep working! During this phase, the focus duration w

# Delete Assistant

In [111]:
response = client.beta.assistants.delete(assistant1.id)
time.sleep(3)
print(response)

AssistantDeleted(id='asst_AwF9iI3swIM68ot57sSha8Is', deleted=True, object='assistant.deleted')


In [112]:
response = client.beta.assistants.delete(assistant2.id)
time.sleep(3)
print(response)

AssistantDeleted(id='asst_pdVd0i9kFnLljlsxW5U8gwu9', deleted=True, object='assistant.deleted')
